In [2]:
import os
import glob
import json
import pandas as pd
def read_recursive_parquet(root_dir):
    all_files = glob.glob(os.path.join(root_dir, '**', '*.parquet'), recursive=True)
    print(all_files)
    df = pd.concat((pd.read_parquet(f) for f in all_files), ignore_index=True)
    return df

cohort_dir = "/data/processed_datasets/processed_datasets/mimic/mimic_3.1/patient_outcome_tasks/in_hospital_mortality"
# cohort = read_recursive_parquet(cohort_dir)
# print(cohort.prediction_time)
label_name = os.path.basename(os.path.normpath(cohort_dir))
print(label_name)

in_hospital_mortality


In [3]:
import pandas as pd

# phenotype_path = "/data/processed_datasets/processed_datasets/mimic/phenotype_task"
phenotype_path = "/user/zj2398/cache/mimic/mimic-3.1-meds/phenotype_task/"
all_files = glob.glob(os.path.join(phenotype_path, '**', '*.parquet'), recursive=True)
for file_path in all_files:
    disease_df = pd.read_parquet(file_path)
    disease_df = disease_df.rename(columns={"binary_label":"boolean_value"})
    disease_df.to_parquet(file_path)

In [ ]:
# fetch tte labels from phenotype task to processed task
from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import femr
import femr.featurizers
import femr.splits

diseases = ["stroke","masld"]
# phenotype_dir = Path("/data/processed_datasets/processed_datasets/mimic/phenotype_task")
# file1 = pd.read_parquet('/data/processed_datasets/processed_datasets/mimic/phenotype_task/celiac/celiac_cohort.parquet')
phenotype_dir = Path("/user/zj2398/cache/mimic/mimic-3.1-meds/phenotype_task")
embedding_dir = Path("/user/zj2398/cache/motor_mimic_8k/results")
for disease in diseases:
    df1 = pd.read_parquet(phenotype_dir / disease / f"{disease}_cohort.parquet")
    # print(cohort_label.head())

    for split in ["train", "test"]:
        split_path = embedding_dir / disease/ f"motor/features_with_label/{split}.parquet"
        with open(split_path, 'rb') as f:
            df2 = pd.read_parquet(f)
        print(df2)


        # --- 0) (Optional) make copies so originals stay intact
        df1_ = df1.copy()
        df2_ = df2.copy()

        # --- 1) Ensure dtypes match exactly
        # prediction_time to datetime (naive)
        df1_["prediction_time"] = pd.to_datetime(df1_["prediction_time"], errors="raise", utc=False)
        df2_["prediction_time"] = pd.to_datetime(df2_["prediction_time"], errors="raise", utc=False)

        # boolean_value to proper bool
        # (handles cases where it's already bool or string "True"/"False")
        df1_["boolean_value"] = df1_["boolean_value"].astype(str).str.lower().map({"true": True, "false": False})
        df2_["boolean_value"] = df2_["boolean_value"].astype(str).str.lower().map({"true": True, "false": False})

        # subject_id to consistent integer (or string if you prefer)
        df1_["subject_id"] = pd.to_numeric(df1_["subject_id"], downcast="integer")
        df2_["subject_id"] = pd.to_numeric(df2_["subject_id"], downcast="integer")

        # --- 2) Check that the join key is unique on the right side (df1)
        key_cols = ["subject_id", "prediction_time", "boolean_value"]
        dupes = df1_.duplicated(subset=key_cols, keep=False)
        if dupes.any():
            # Show a few offending keys to help diagnose
            raise ValueError(
                "df1 has duplicate rows for the join key; expected 1 row per key.\n"
                f"Examples:\n{df1_.loc[dupes, key_cols].head(10)}"
            )

        # --- 3) Merge (df2 left-join to df1), enforcing m:1 cardinality
        merged = df2_.merge(
            df1_[key_cols + ["time_to_event_days"]],
            on=key_cols,
            how="left",
            validate="m:1"
        )

        # --- 4) Verify every df2 row matched something in df1
        num_unmatched = merged["time_to_event_days"].isna().sum()
        if num_unmatched:
            # Show a few missing keys to debug
            missing_keys = merged.loc[merged["time_to_event_days"].isna(), key_cols].drop_duplicates().head(10)
            raise ValueError(
                f"{num_unmatched} row(s) in df2 did not find a match in df1. "
                f"Examples of missing keys:\n{missing_keys}"
            )

        # --- 5) If you only want df2 + the added label:
        df2_with_label = merged  # same columns as df2, plus time_to_event_days
        # print(df2_with_label)
        df2_with_label.to_parquet(split_path)








        subject_id     prediction_time  boolean_value  \
0         12754657 2142-03-24 19:57:00          False   
1         10009326 2116-03-31 00:31:00          False   
2         10722703 2132-06-11 05:07:00          False   
3         17578973 2168-12-14 03:44:00          False   
4         13080738 2133-07-31 20:17:00          False   
...            ...                 ...            ...   
281658    14244279 2146-02-02 20:41:00          False   
281659    19168752 2189-11-26 16:15:00          False   
281660    14685940 2127-07-31 14:46:00          False   
281661    15229574 2125-03-10 23:37:00          False   
281662    14313330 2147-05-04 21:20:00          False   

                                                 features  time_to_event_days  
0       [-1.0309983, 1.1677849, -0.30635804, 0.0384228...          251.168750  
1       [1.1620811, 1.5489517, -2.0962741, 1.7200605, ...            9.978472  
2       [0.3266717, 1.5968416, -0.8996342, 0.48808944,...         1378.4026